In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv('data_sample.csv')

# set time column to be 2 seconds apart
df['t'] = np.arange(0, 2*len(df), 2)

df.describe()

,current,voltage,soc,t
count,86377.000000,86377.000000,86377.000000,86377.000000
mean,0.655906,1004.405162,65.222531,86376.000000
std,17.144998,50.034648,22.792818,49870.072876
min,-74.135154,859.146629,2.800000,0.000000
25%,-0.200000,969.319987,51.400000,43188.000000
50%,0.000000,990.500000,59.600000,86376.000000
75%,0.000000,1044.000000,86.000000,129564.000000
max,61.200000,1087.920681,97.700000,172752.000000


In [15]:
def c_rate(df: pd.DataFrame, capacity: float) -> float:
    C = capacity * 3600
    I = df['current'].abs()
    dSoC = df['soc'].diff().abs() / 100
    
    return (I.sum() / C) * (1 / dSoC.sum())

In [16]:
c_rate(df=df, capacity=90)

0.5323975331531735

In [17]:
def rte(df: pd.DataFrame) -> float:
    cols = ['soc', 'current', 'voltage']
    dt = df['t'].diff() / 3600 # seconds in hours
    
    charge = df[df['current'] > 0][cols]
    dSoC_charge = charge['soc'].diff().abs()
    charged_energy = charge['current'] * charge['voltage'] * dt

    discharge = df[df['current'] < 0][cols]
    dSoC_discharge = discharge['soc'].diff().abs()
    discharged_energy = discharge['current'].abs() * discharge['voltage'] * dt

    return (discharged_energy.sum() / charged_energy.sum()) * (dSoC_charge.sum() / dSoC_discharge.sum())

In [18]:
rte(df)

0.7978238199725021